<a href="https://colab.research.google.com/github/orutkina/-./blob/main/%D0%94%D0%974____4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas seaborn scikit-learn

# Домашнее задание 4

In [ ]:
# Установка необходимых библиотек
!pip install pandas seaborn scikit-learn matplotlib plotly

# Импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

Часть 1: Загрузка и предварительная обработка данных

In [ ]:
print("="*80)
print("ЧАСТЬ 1: ЗАГРУЗКА И ПРЕДВАРИТЕЛЬНАЯ ОБРАБОТКА ДАННЫХ")
print("="*80)

# Загрузка данных
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')

# Создаем копию для анализа (необработанные данные)
df_not_processed = df.copy()

print(f"Размер исходного датасета: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")

# Просмотр первых строк
print("\nПервые 5 строк данных:")
print(df.head())

# Информация о типах данных
print("\nИнформация о типах данных:")
print(df.info())

# Проверка на пропущенные значения
print("\nПропущенные значения:")
print(df.isnull().sum())

Часть 2: Обработка категориальных признаков

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 2: ОБРАБОТКА КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ")
print("="*80)

# Определяем категориальные признаки
categorical_cols = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']

# Преобразуем в тип category
for col in categorical_cols:
    df[col] = df[col].astype('category')
    df_not_processed[col] = df_not_processed[col].astype('category')

print("Категориальные признаки преобразованы в тип 'category'")
print(f"Категориальные колонки: {categorical_cols}")

# Применяем One-Hot Encoding к категориальным признакам
print("\nПрименяем One-Hot Encoding...")

# Используем pd.get_dummies с drop_first=True для избежания дамми-ловушки
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(f"Размер датасета после One-Hot Encoding: {df_encoded.shape}")
print(f"Новые колонки (первые 10): {df_encoded.columns.tolist()[:10]}")
print(f"... и еще {len(df_encoded.columns.tolist()[10:])} колонок")

# Сохраняем имена числовых признаков для масштабирования
numeric_features = df_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()
print(f"\nЧисловые признаки для масштабирования ({len(numeric_features)}):")
print(numeric_features)

Часть 3: Масштабирование данных

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 3: МАСШТАБИРОВАНИЕ ДАННЫХ")
print("="*80)

# Создаем копию для масштабирования
df_scaled = df_encoded.copy()

# Используем StandardScaler для масштабирования (центрирование и приведение к единичной дисперсии)
scaler = StandardScaler()
df_scaled[numeric_features] = scaler.fit_transform(df_scaled[numeric_features])

print("Данные масштабированы с использованием StandardScaler")
print("\nСтатистика после масштабирования (первые 5 строк):")
print(df_scaled[numeric_features].head())

# Также создадим версию с MinMaxScaler для сравнения
df_minmax = df_encoded.copy()
minmax_scaler = MinMaxScaler()
df_minmax[numeric_features] = minmax_scaler.fit_transform(df_minmax[numeric_features])

print("\nДля сравнения создана также версия с MinMaxScaler")

# Для кластеризации будем использовать StandardScaler, так как он лучше подходит для алгоритмов,
# основанных на расстояниях (K-means, DBSCAN)

Часть 4: K-means кластеризация

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 4: K-MEANS КЛАСТЕРИЗАЦИЯ")
print("="*80)

# Выбираем данные для кластеризации (только числовые признаки)
X = df_scaled[numeric_features].values

print(f"Размерность данных для кластеризации: {X.shape}")
print(f"Количество признаков: {X.shape[1]}")

# Сначала попробуем K-means с 3 кластерами (исходя из EDA в предыдущих заданиях)
kmeans_3 = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters_3 = kmeans_3.fit_predict(X)

# Добавляем кластеры в исходный датасет
df_not_processed['cluster_kmeans_3'] = clusters_3
df_scaled['cluster_kmeans_3'] = clusters_3

print("K-means кластеризация с 3 кластерами выполнена")
print(f"Метки кластеров: {np.unique(clusters_3)}")
print(f"Размеры кластеров:")
for i in range(3):
    count = np.sum(clusters_3 == i)
    print(f"  Кластер {i}: {count} наблюдений ({count/len(clusters_3)*100:.1f}%)")

# Вычисляем метрики качества
silhouette_3 = silhouette_score(X, clusters_3)
calinski_3 = calinski_harabasz_score(X, clusters_3)
davies_3 = davies_bouldin_score(X, clusters_3)

print("\nМетрики качества для 3 кластеров:")
print(f"  Silhouette Score: {silhouette_3:.4f} (чем ближе к 1, тем лучше)")
print(f"  Calinski-Harabasz Index: {calinski_3:.2f} (чем выше, тем лучше)")
print(f"  Davies-Bouldin Index: {davies_3:.4f} (чем ближе к 0, тем лучше)")

Часть 5: Визуализация K-means кластеризации

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 5: ВИЗУАЛИЗАЦИЯ K-MEANS КЛАСТЕРИЗАЦИИ")
print("="*80)

# Для визуализации используем PCA для уменьшения размерности до 2D
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X)

print(f"Объясненная дисперсия PCA: {pca.explained_variance_ratio_}")
print(f"Суммарная объясненная дисперсия: {sum(pca.explained_variance_ratio_):.3f}")

# Создаем графики
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Визуализация кластеров в пространстве PCA
scatter = axes[0, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=clusters_3, cmap='viridis', alpha=0.7)
axes[0, 0].set_xlabel('PCA Component 1')
axes[0, 0].set_ylabel('PCA Component 2')
axes[0, 0].set_title('K-means (3 кластера) - PCA проекция')
plt.colorbar(scatter, ax=axes[0, 0])

# 2. Центроиды кластеров
centroids_pca = pca.transform(kmeans_3.cluster_centers_)
axes[0, 1].scatter(X_pca[:, 0], X_pca[:, 1], c=clusters_3, cmap='viridis', alpha=0.3)
axes[0, 1].scatter(centroids_pca[:, 0], centroids_pca[:, 1],
                   marker='X', s=200, c='red', label='Центроиды')
axes[0, 1].set_xlabel('PCA Component 1')
axes[0, 1].set_ylabel('PCA Component 2')
axes[0, 1].set_title('Центроиды кластеров')
axes[0, 1].legend()

# 3. Распределение кластеров
cluster_counts = df_not_processed['cluster_kmeans_3'].value_counts().sort_index()
bars = axes[0, 2].bar(cluster_counts.index, cluster_counts.values)
axes[0, 2].set_xlabel('Кластер')
axes[0, 2].set_ylabel('Количество наблюдений')
axes[0, 2].set_title('Распределение по кластерам')
for bar in bars:
    height = bar.get_height()
    axes[0, 2].text(bar.get_x() + bar.get_width()/2., height + 2,
                   f'{height}', ha='center', va='bottom')

# 4. Анализ важных признаков для кластеров
# Вычисляем средние значения признаков по кластерам
cluster_means = df_scaled.groupby('cluster_kmeans_3')[numeric_features].mean()

# Выбираем топ-5 признаков с наибольшей вариацией между кластерами
feature_variation = cluster_means.std().sort_values(ascending=False).head(5)
top_features = feature_variation.index.tolist()

# Визуализируем средние значения топ-признаков по кластерам
x = np.arange(len(top_features))
width = 0.25

for i in range(3):
    values = cluster_means.loc[i, top_features].values
    axes[1, 0].bar(x + i*width - width, values, width, label=f'Кластер {i}')

axes[1, 0].set_xlabel('Признаки')
axes[1, 0].set_ylabel('Среднее значение (стандартизированное)')
axes[1, 0].set_title('Средние значения топ-5 признаков по кластерам')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(top_features, rotation=45, ha='right')
axes[1, 0].legend()

# 5. Визуализация с t-SNE (альтернатива PCA)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X)

scatter_tsne = axes[1, 1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=clusters_3, cmap='viridis', alpha=0.7)
axes[1, 1].set_xlabel('t-SNE Component 1')
axes[1, 1].set_ylabel('t-SNE Component 2')
axes[1, 1].set_title('K-means (3 кластера) - t-SNE проекция')
plt.colorbar(scatter_tsne, ax=axes[1, 1])

# 6. Анализ распределения исходных категориальных признаков по кластерам
# Например, распределение по полу
gender_dist = pd.crosstab(df_not_processed['cluster_kmeans_3'], df_not_processed['Gender'], normalize='index')
gender_dist.plot(kind='bar', stacked=True, ax=axes[1, 2])
axes[1, 2].set_xlabel('Кластер')
axes[1, 2].set_ylabel('Доля')
axes[1, 2].set_title('Распределение по полу в кластерах')
axes[1, 2].legend(title='Пол')
axes[1, 2].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

# Анализ кластеров
print("\nАНАЛИЗ КЛАСТЕРОВ K-MEANS (3 КЛАСТЕРА):")
print("="*50)

# Для каждого кластера выведем характеристику
for cluster_num in range(3):
    cluster_data = df_not_processed[df_not_processed['cluster_kmeans_3'] == cluster_num]

    print(f"\nКЛАСТЕР {cluster_num} ({len(cluster_data)} наблюдений, {len(cluster_data)/len(df_not_processed)*100:.1f}%):")

    # Средние значения ключевых числовых признаков
    print("  Средние значения ключевых признаков:")
    key_features = ['Age', 'Sleep Duration', 'Physical Activity Level', 'Stress Level', 'Heart Rate']
    for feature in key_features:
        mean_val = cluster_data[feature].mean()
        print(f"    {feature}: {mean_val:.2f}")

    # Распределение по категориальным признакам
    print("  Распределение по категориальным признакам:")

    # Пол
    gender_dist = cluster_data['Gender'].value_counts(normalize=True)
    print(f"    Пол: {gender_dist.idxmax()} ({gender_dist.max()*100:.1f}%)")

    # Категория BMI
    if 'BMI Category' in cluster_data.columns:
        bmi_dist = cluster_data['BMI Category'].value_counts(normalize=True)
        if not bmi_dist.empty:
            print(f"    Категория BMI: {bmi_dist.idxmax()} ({bmi_dist.max()*100:.1f}%)")

    # Расстройство сна
    if 'Sleep Disorder' in cluster_data.columns:
        disorder_dist = cluster_data['Sleep Disorder'].value_counts(normalize=True)
        if not disorder_dist.empty:
            print(f"    Расстройство сна: {disorder_dist.idxmax()} ({disorder_dist.max()*100:.1f}%)")

Часть 6: Метод локтя для определения оптимального числа кластеров

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 6: МЕТОД ЛОКТЯ ДЛЯ ОПРЕДЕЛЕНИЯ ОПТИМАЛЬНОГО ЧИСЛА КЛАСТЕРОВ")
print("="*80)

# Используем метод локтя для определения оптимального числа кластеров
inertia = []
silhouette_scores = []
calinski_scores = []
davies_scores = []

# Пробуем разное количество кластеров от 2 до 10
k_range = range(2, 11)

print("Вычисляем метрики для разного количества кластеров...")
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X)

    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X, clusters))
    calinski_scores.append(calinski_harabasz_score(X, clusters))
    davies_scores.append(davies_bouldin_score(X, clusters))

    print(f"  k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores[-1]:.4f}")

# Визуализируем результаты
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Метод локтя (Inertia)
axes[0, 0].plot(k_range, inertia, marker='o')
axes[0, 0].set_xlabel('Количество кластеров (k)')
axes[0, 0].set_ylabel('Inertia (сумма квадратов расстояний)')
axes[0, 0].set_title('Метод локтя для K-means')
axes[0, 0].grid(True, alpha=0.3)

# Добавляем локти (изгибы) для наглядности
# Вычисляем разности второго порядка для нахождения "локтя"
differences = np.diff(inertia)
second_diff = np.diff(differences)
elbow_point = np.argmin(second_diff) + 2  # +2 потому что мы потеряли два элемента

axes[0, 0].axvline(x=elbow_point, color='r', linestyle='--', alpha=0.7,
                   label=f'Предполагаемый локоть: k={elbow_point}')
axes[0, 0].legend()

# 2. Silhouette Score
axes[0, 1].plot(k_range, silhouette_scores, marker='o', color='green')
axes[0, 1].set_xlabel('Количество кластеров (k)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score для разного k')
axes[0, 1].grid(True, alpha=0.3)

# Находим k с максимальным Silhouette Score
best_k_silhouette = k_range[np.argmax(silhouette_scores)]
axes[0, 1].axvline(x=best_k_silhouette, color='r', linestyle='--', alpha=0.7,
                   label=f'Лучший k по Silhouette: {best_k_silhouette}')
axes[0, 1].legend()

# 3. Calinski-Harabasz Index
axes[1, 0].plot(k_range, calinski_scores, marker='o', color='orange')
axes[1, 0].set_xlabel('Количество кластеров (k)')
axes[1, 0].set_ylabel('Calinski-Harabasz Index')
axes[1, 0].set_title('Calinski-Harabasz Index для разного k')
axes[1, 0].grid(True, alpha=0.3)

# Находим k с максимальным Calinski-Harabasz
best_k_calinski = k_range[np.argmax(calinski_scores)]
axes[1, 0].axvline(x=best_k_calinski, color='r', linestyle='--', alpha=0.7,
                   label=f'Лучший k по Calinski: {best_k_calinski}')
axes[1, 0].legend()

# 4. Davies-Bouldin Index
axes[1, 1].plot(k_range, davies_scores, marker='o', color='purple')
axes[1, 1].set_xlabel('Количество кластеров (k)')
axes[1, 1].set_ylabel('Davies-Bouldin Index')
axes[1, 1].set_title('Davies-Bouldin Index для разного k')
axes[1, 1].grid(True, alpha=0.3)

# Находим k с минимальным Davies-Bouldin
best_k_davies = k_range[np.argmin(davies_scores)]
axes[1, 1].axvline(x=best_k_davies, color='r', linestyle='--', alpha=0.7,
                   label=f'Лучший k по Davies: {best_k_davies}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Анализируем результаты
print("\nАНАЛИЗ МЕТОДА ЛОКТЯ:")
print(f"1. Метод локтя предполагает оптимальное k = {elbow_point}")
print(f"2. Лучший k по Silhouette Score: {best_k_silhouette}")
print(f"3. Лучший k по Calinski-Harabasz Index: {best_k_calinski}")
print(f"4. Лучший k по Davies-Bouldin Index: {best_k_davies}")

# Выбираем оптимальное k на основе всех метрик
# Часто выбирают k, где Silhouette Score максимален, а Inertia имеет явный изгиб
optimal_k = best_k_silhouette  # Выбираем на основе Silhouette Score
print(f"\nВыбираем оптимальное k = {optimal_k} (на основе Silhouette Score)")

# Запускаем K-means с оптимальным k
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters_optimal = kmeans_optimal.fit_predict(X)

# Добавляем кластеры в датасеты
df_not_processed['cluster_kmeans_optimal'] = clusters_optimal
df_scaled['cluster_kmeans_optimal'] = clusters_optimal

print(f"\nK-means с оптимальным k={optimal_k} выполнен")
print("Размеры кластеров:")
for i in range(optimal_k):
    count = np.sum(clusters_optimal == i)
    print(f"  Кластер {i}: {count} наблюдений ({count/len(clusters_optimal)*100:.1f}%)")

# Вычисляем метрики
silhouette_opt = silhouette_score(X, clusters_optimal)
calinski_opt = calinski_harabasz_score(X, clusters_optimal)
davies_opt = davies_bouldin_score(X, clusters_optimal)

print("\nМетрики качества для оптимального k:")
print(f"  Silhouette Score: {silhouette_opt:.4f}")
print(f"  Calinski-Harabasz Index: {calinski_opt:.2f}")
print(f"  Davies-Bouldin Index: {davies_opt:.4f}")

Часть 7: DBSCAN кластеризация

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 7: DBSCAN КЛАСТЕРИЗАЦИЯ")
print("="*80)

print("DBSCAN (Density-Based Spatial Clustering of Applications with Noise)")
print("Особенности DBSCAN:")
print("- Не требует задания числа кластеров заранее")
print("- Может находить кластеры произвольной формы")
print("- Помечает выбросы как шум (кластер -1)")
print("- Чувствителен к параметрам eps и min_samples")

# Подбираем оптимальные параметры для DBSCAN
# Используем метод k-distance графика для выбора eps

from sklearn.neighbors import NearestNeighbors

# Вычисляем расстояния до k ближайших соседей
k = 5  # min_samples обычно выбирают как 2*dim, где dim - размерность данных
nn = NearestNeighbors(n_neighbors=k)
nn.fit(X)
distances, indices = nn.kneighbors(X)

# Сортируем расстояния до k-го соседа
k_distances = np.sort(distances[:, k-1])

# Строим график k-distance
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(k_distances)+1), k_distances)
plt.xlabel('Точки, отсортированные по расстоянию до k-го соседа')
plt.ylabel(f'Расстояние до {k}-го соседа')
plt.title('K-distance график для выбора eps')
plt.grid(True, alpha=0.3)

# Ищем "локоть" на графике
# Вычисляем вторую производную для нахождения точки изгиба
gradients = np.gradient(k_distances)
second_gradients = np.gradient(gradients)
elbow_idx = np.argmax(second_gradients) + 1
eps_suggestion = k_distances[elbow_idx]

plt.axhline(y=eps_suggestion, color='r', linestyle='--',
           label=f'Предлагаемое eps: {eps_suggestion:.3f}')
plt.axvline(x=elbow_idx, color='r', linestyle='--', alpha=0.5)
plt.legend()
plt.show()

print(f"\nНа основе k-distance графика предлагается eps ≈ {eps_suggestion:.3f}")
print(f"min_samples обычно выбирают как 2 * размерность данных = {2 * X.shape[1]}")

# Пробуем разные параметры
param_combinations = [
    {'eps': 3.0, 'min_samples': 10},
    {'eps': 3.5, 'min_samples': 10},
    {'eps': 4.0, 'min_samples': 10},
    {'eps': 4.0, 'min_samples': 15},
    {'eps': 4.5, 'min_samples': 15},
    {'eps': 5.0, 'min_samples': 15},
]

print("\nПробуем разные комбинации параметров DBSCAN:")

best_score = -1
best_params = None
best_clusters = None

results = []

for params in param_combinations:
    dbscan = DBSCAN(eps=params['eps'], min_samples=params['min_samples'])
    clusters = dbscan.fit_predict(X)

    # Количество кластеров (исключая шум -1)
    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)

    # Процент шума
    noise_ratio = np.sum(clusters == -1) / len(clusters) * 100

    # Вычисляем метрики только если есть хотя бы 2 кластера и не все точки - шум
    if n_clusters >= 2 and noise_ratio < 100:
        # Для DBSCAN иногда сложно вычислить метрики из-за шума
        # Берем только точки, которые не являются шумом
        non_noise_mask = clusters != -1
        if np.sum(non_noise_mask) > 1:  # Нужно хотя бы 2 точки не-шума
            silhouette = silhouette_score(X[non_noise_mask], clusters[non_noise_mask])
            calinski = calinski_harabasz_score(X[non_noise_mask], clusters[non_noise_mask])
            davies = davies_bouldin_score(X[non_noise_mask], clusters[non_noise_mask])
        else:
            silhouette = calinski = davies = np.nan
    else:
        silhouette = calinski = davies = np.nan

    results.append({
        'eps': params['eps'],
        'min_samples': params['min_samples'],
        'n_clusters': n_clusters,
        'noise_ratio': noise_ratio,
        'silhouette': silhouette,
        'calinski': calinski,
        'davies': davies
    })

    print(f"  eps={params['eps']}, min_samples={params['min_samples']}: "
          f"{n_clusters} кластеров, шум: {noise_ratio:.1f}%")

# Создаем DataFrame с результатами
results_df = pd.DataFrame(results)

print("\nРезультаты подбора параметров DBSCAN:")
print(results_df.to_string(index=False))

# Выбираем лучшие параметры на основе Silhouette Score
# Игнорируем варианты с NaN
valid_results = results_df[~results_df['silhouette'].isna()]
if not valid_results.empty:
    best_idx = valid_results['silhouette'].idxmax()
    best_params = {
        'eps': valid_results.loc[best_idx, 'eps'],
        'min_samples': valid_results.loc[best_idx, 'min_samples']
    }
else:
    # Если все результаты с NaN, берем с наименьшим процентом шума и хотя бы 2 кластера
    valid_results = results_df[results_df['n_clusters'] >= 2]
    if not valid_results.empty:
        best_idx = valid_results['noise_ratio'].idxmin()
        best_params = {
            'eps': valid_results.loc[best_idx, 'eps'],
            'min_samples': valid_results.loc[best_idx, 'min_samples']
        }
    else:
        # Если ничего не подошло, берем первый вариант
        best_params = param_combinations[0]

print(f"\nВыбранные параметры: eps={best_params['eps']}, min_samples={best_params['min_samples']}")

# Запускаем DBSCAN с выбранными параметрами
dbscan_best = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
clusters_dbscan = dbscan_best.fit_predict(X)

# Добавляем кластеры в датасеты
df_not_processed['cluster_dbscan'] = clusters_dbscan
df_scaled['cluster_dbscan'] = clusters_dbscan

# Анализируем результаты
unique_clusters = np.unique(clusters_dbscan)
n_clusters = len(unique_clusters) - (1 if -1 in unique_clusters else 0)
noise_count = np.sum(clusters_dbscan == -1)
noise_ratio = noise_count / len(clusters_dbscan) * 100

print(f"\nDBSCAN кластеризация завершена:")
print(f"  Всего уникальных меток: {unique_clusters}")
print(f"  Количество кластеров (без шума): {n_clusters}")
print(f"  Количество точек шума: {noise_count} ({noise_ratio:.1f}%)")

if n_clusters >= 2:
    # Вычисляем метрики для не-шумовых точек
    non_noise_mask = clusters_dbscan != -1
    if np.sum(non_noise_mask) > 1:
        silhouette_db = silhouette_score(X[non_noise_mask], clusters_dbscan[non_noise_mask])
        calinski_db = calinski_harabasz_score(X[non_noise_mask], clusters_dbscan[non_noise_mask])
        davies_db = davies_bouldin_score(X[non_noise_mask], clusters_dbscan[non_noise_mask])

        print(f"\nМетрики качества DBSCAN (без шума):")
        print(f"  Silhouette Score: {silhouette_db:.4f}")
        print(f"  Calinski-Harabasz Index: {calinski_db:.2f}")
        print(f"  Davies-Bouldin Index: {davies_db:.4f}")
    else:
        print("Недостаточно не-шумовых точек для вычисления метрик")
else:
    print("DBSCAN нашел менее 2 кластеров, метрики не вычисляются")

Часть 8: Визуализация и анализ DBSCAN кластеризации

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 8: ВИЗУАЛИЗАЦИЯ И АНАЛИЗ DBSCAN КЛАСТЕРИЗАЦИИ")
print("="*80)

# Визуализируем результаты DBSCAN
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. DBSCAN в пространстве PCA (все точки)
scatter_dbscan = axes[0, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=clusters_dbscan,
                                    cmap='tab20', alpha=0.7)
axes[0, 0].set_xlabel('PCA Component 1')
axes[0, 0].set_ylabel('PCA Component 2')
axes[0, 0].set_title(f'DBSCAN кластеризация (eps={best_params["eps"]}, min_samples={best_params["min_samples"]})')
plt.colorbar(scatter_dbscan, ax=axes[0, 0])

# 2. DBSCAN в пространстве PCA (шум отдельно)
# Создаем специальную цветовую схему: шум - серый, кластеры - разные цвета
dbscan_colors = []
color_map = plt.cm.tab20
for cluster in clusters_dbscan:
    if cluster == -1:
        dbscan_colors.append((0.5, 0.5, 0.5, 0.5))  # Серый для шума
    else:
        dbscan_colors.append(color_map(cluster % 20))

axes[0, 1].scatter(X_pca[:, 0], X_pca[:, 1], c=dbscan_colors, alpha=0.7)
axes[0, 1].set_xlabel('PCA Component 1')
axes[0, 1].set_ylabel('PCA Component 2')
axes[0, 1].set_title('DBSCAN (шум серым)')

# 3. Распределение кластеров DBSCAN
dbscan_counts = df_not_processed['cluster_dbscan'].value_counts().sort_index()
bars_dbscan = axes[0, 2].bar(dbscan_counts.index, dbscan_counts.values)
axes[0, 2].set_xlabel('Кластер ( -1 = шум )')
axes[0, 2].set_ylabel('Количество наблюдений')
axes[0, 2].set_title('Распределение по кластерам DBSCAN')

# Подписываем количество на каждом столбце
for bar in bars_dbscan:
    height = bar.get_height()
    axes[0, 2].text(bar.get_x() + bar.get_width()/2., height + 2,
                   f'{int(height)}', ha='center', va='bottom', fontsize=9)

# 4. Сравнение K-means и DBSCAN
# Выделяем шум от DBSCAN на K-means кластеризации
noise_mask = clusters_dbscan == -1
axes[1, 0].scatter(X_pca[~noise_mask, 0], X_pca[~noise_mask, 1],
                   c=clusters_optimal[~noise_mask], cmap='viridis', alpha=0.7, label='K-means кластеры')
axes[1, 0].scatter(X_pca[noise_mask, 0], X_pca[noise_mask, 1],
                   c='red', marker='x', s=50, label='DBSCAN шум')
axes[1, 0].set_xlabel('PCA Component 1')
axes[1, 0].set_ylabel('PCA Component 2')
axes[1, 0].set_title('K-means кластеры с DBSCAN шумом (красные X)')
axes[1, 0].legend()

# 5. DBSCAN в пространстве t-SNE
scatter_dbscan_tsne = axes[1, 1].scatter(X_tsne[:, 0], X_tsne[:, 1],
                                         c=clusters_dbscan, cmap='tab20', alpha=0.7)
axes[1, 1].set_xlabel('t-SNE Component 1')
axes[1, 1].set_ylabel('t-SNE Component 2')
axes[1, 1].set_title('DBSCAN - t-SNE проекция')

# 6. Анализ характеристик шумовых точек
if noise_count > 0:
    noise_data = df_not_processed[df_not_processed['cluster_dbscan'] == -1]
    non_noise_data = df_not_processed[df_not_processed['cluster_dbscan'] != -1]

    # Сравним средние значения ключевых признаков
    key_features = ['Age', 'Sleep Duration', 'Physical Activity Level', 'Stress Level']

    noise_means = noise_data[key_features].mean()
    non_noise_means = non_noise_data[key_features].mean()

    x = np.arange(len(key_features))
    width = 0.35

    axes[1, 2].bar(x - width/2, noise_means, width, label='Шумовые точки', alpha=0.7)
    axes[1, 2].bar(x + width/2, non_noise_means, width, label='Кластеризованные точки', alpha=0.7)

    axes[1, 2].set_xlabel('Признаки')
    axes[1, 2].set_ylabel('Среднее значение')
    axes[1, 2].set_title('Сравнение шумовых и кластеризованных точек')
    axes[1, 2].set_xticks(x)
    axes[1, 2].set_xticklabels(key_features, rotation=45, ha='right')
    axes[1, 2].legend()
else:
    axes[1, 2].text(0.5, 0.5, 'Нет шумовых точек\nв DBSCAN кластеризации',
                    ha='center', va='center', fontsize=12)
    axes[1, 2].set_title('Анализ шумовых точек')

plt.tight_layout()
plt.show()

# Анализ DBSCAN кластеров
print("\nАНАЛИЗ DBSCAN КЛАСТЕРОВ:")
print("="*50)

if n_clusters > 0:
    for cluster_num in sorted([c for c in unique_clusters if c != -1]):
        cluster_data = df_not_processed[df_not_processed['cluster_dbscan'] == cluster_num]

        print(f"\nКЛАСТЕР {cluster_num} ({len(cluster_data)} наблюдений, {len(cluster_data)/len(df_not_processed)*100:.1f}%):")

        # Средние значения ключевых числовых признаков
        print("  Средние значения ключевых признаков:")
        key_features = ['Age', 'Sleep Duration', 'Physical Activity Level', 'Stress Level']
        for feature in key_features:
            mean_val = cluster_data[feature].mean()
            print(f"    {feature}: {mean_val:.2f}")

        # Распределение по полу
        if len(cluster_data) > 0:
            gender_dist = cluster_data['Gender'].value_counts(normalize=True)
            if not gender_dist.empty:
                print(f"    Пол: {gender_dist.idxmax()} ({gender_dist.max()*100:.1f}%)")
else:
    print("DBSCAN не обнаружил кластеров (только шум)")

# Анализ шумовых точек
if noise_count > 0:
    print(f"\nАНАЛИЗ ШУМОВЫХ ТОЧЕК ({noise_count} точек, {noise_ratio:.1f}%):")
    noise_data = df_not_processed[df_not_processed['cluster_dbscan'] == -1]

    print("  Средние значения ключевых признаков:")
    for feature in key_features:
        mean_val = noise_data[feature].mean()
        print(f"    {feature}: {mean_val:.2f}")

    # Проверяем, в какие K-means кластеры попадают шумовые точки
    print(f"  Распределение по K-means кластерам (k={optimal_k}):")
    kmeans_dist = noise_data['cluster_kmeans_optimal'].value_counts()
    for k_cluster, count in kmeans_dist.items():
        print(f"    K-means кластер {k_cluster}: {count} точек ({count/len(noise_data)*100:.1f}%)")

Часть 9: Сравнение методов кластеризации

In [ ]:
print("\n" + "="*80)
print("ЧАСТЬ 9: СРАВНЕНИЕ МЕТОДОВ КЛАСТЕРИЗАЦИИ")
print("="*80)

# Сравниваем K-means и DBSCAN
comparison_data = []

# 1. K-means с 3 кластерами
comparison_data.append({
    'Метод': 'K-means (k=3)',
    'Количество кластеров': 3,
    'Процент шума': 0.0,
    'Silhouette Score': silhouette_3,
    'Calinski-Harabasz': calinski_3,
    'Davies-Bouldin': davies_3
})

# 2. K-means с оптимальным k
comparison_data.append({
    'Метод': f'K-means (оптимальное k={optimal_k})',
    'Количество кластеров': optimal_k,
    'Процент шума': 0.0,
    'Silhouette Score': silhouette_opt,
    'Calinski-Harabasz': calinski_opt,
    'Davies-Bouldin': davies_opt
})

# 3. DBSCAN
if n_clusters >= 2 and 'silhouette_db' in locals():
    comparison_data.append({
        'Метод': f'DBSCAN (eps={best_params["eps"]}, min_samples={best_params["min_samples"]})',
        'Количество кластеров': n_clusters,
        'Процент шума': noise_ratio,
        'Silhouette Score': silhouette_db,
        'Calinski-Harabasz': calinski_db,
        'Davies-Bouldin': davies_db
    })

# Создаем DataFrame для сравнения
comparison_df = pd.DataFrame(comparison_data)
print("\nСРАВНЕНИЕ МЕТОДОВ КЛАСТЕРИЗАЦИИ:")
print(comparison_df.to_string(index=False))

# Визуализируем сравнение
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Сравнение количества кластеров
methods = comparison_df['Метод'].tolist()
n_clusters_list = comparison_df['Количество кластеров'].tolist()

bars1 = axes[0, 0].bar(methods, n_clusters_list)
axes[0, 0].set_ylabel('Количество кластеров')
axes[0, 0].set_title('Сравнение количества кластеров')
axes[0, 0].tick_params(axis='x', rotation=15)
for bar in bars1:
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.1,
                   f'{int(height)}', ha='center', va='bottom')

# 2. Сравнение Silhouette Score
silhouette_list = comparison_df['Silhouette Score'].tolist()

bars2 = axes[0, 1].bar(methods, silhouette_list, color='green')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Сравнение Silhouette Score (чем выше, тем лучше)')
axes[0, 1].tick_params(axis='x', rotation=15)
axes[0, 1].axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='Хорошая кластеризация')
axes[0, 1].legend()
for bar in bars2:
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{height:.3f}', ha='center', va='bottom')

# 3. Сравнение Calinski-Harabasz Index
calinski_list = comparison_df['Calinski-Harabasz'].tolist()

bars3 = axes[1, 0].bar(methods, calinski_list, color='orange')
axes[1, 0].set_ylabel('Calinski-Harabasz Index')
axes[1, 0].set_title('Сравнение Calinski-Harabasz (чем выше, тем лучше)')
axes[1, 0].tick_params(axis='x', rotation=15)
for bar in bars3:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 5,
                   f'{height:.0f}', ha='center', va='bottom')

# 4. Сравнение Davies-Bouldin Index
davies_list = comparison_df['Davies-Bouldin'].tolist()

bars4 = axes[1, 1].bar(methods, davies_list, color='purple')
axes[1, 1].set_ylabel('Davies-Bouldin Index')
axes[1, 1].set_title('Сравнение Davies-Bouldin (чем ниже, тем лучше)')
axes[1, 1].tick_params(axis='x', rotation=15)
for bar in bars4:
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                   f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Итоговый анализ и выводы
print("\n" + "="*80)
print("ИТОГОВЫЙ АНАЛИЗ И ВЫВОДЫ")
print("="*80)

print("\n1. АНАЛИЗ K-MEANS:")
print("   - Метод локтя показал оптимальное k =", elbow_point)
print(f"   - Silhouette Score максимизируется при k = {best_k_silhouette}")
print(f"   - K-means создает четкие, сферические кластеры одинакового размера")
print(f"   - Не учитывает шумовые точки и выбросы")

print("\n2. АНАЛИЗ DBSCAN:")
print(f"   - Обнаружил {n_clusters} кластеров и {noise_ratio:.1f}% шума")
print(f"   - Может находить кластеры произвольной формы")
print(f"   - Автоматически идентифицирует выбросы как шум")
print(f"   - Чувствителен к параметрам eps и min_samples")

print("\n3. СРАВНЕНИЕ МЕТОДОВ:")
print("   - K-means лучше подходит для данных со сферическими кластерами")
print("   - DBSCAN лучше для данных с кластерами произвольной формы и наличием выбросов")
print("   - K-means требует задания числа кластеров заранее")
print("   - DBSCAN определяет число кластеров автоматически")

print("\n4. РЕКОМЕНДАЦИИ ДЛЯ ДАННОГО ДАТАСЕТА:")

# На основе метрик и визуального анализа делаем вывод
if 'silhouette_db' in locals():
    if silhouette_opt > silhouette_db and silhouette_opt > 0.5:
        print("   - K-means показывает лучшие результаты на данном датасете")
        print(f"   - Silhouette Score K-means ({silhouette_opt:.3f}) > DBSCAN ({silhouette_db:.3f})")
        print("   - Рекомендуется использовать K-means с оптимальным k")
    elif silhouette_db > silhouette_opt and silhouette_db > 0.5:
        print("   - DBSCAN показывает лучшие результаты на данном датасете")
        print(f"   - Silhouette Score DBSCAN ({silhouette_db:.3f}) > K-means ({silhouette_opt:.3f})")
        print("   - Рекомендуется использовать DBSCAN")
    else:
        print("   - Оба метода показывают средние результаты")
        print("   - Возможно, данные плохо кластеризуются или нужны другие методы")
else:
    print("   - K-means показал лучшие результаты (DBSCAN не нашел кластеров или метрики не вычислены)")
    print(f"   - Silhouette Score K-means: {silhouette_opt:.3f}")
    print("   - Рекомендуется использовать K-means")

print("\n5. ПРАКТИЧЕСКИЕ ВЫВОДЫ ДЛЯ АНАЛИЗА СНА:")
print("   - На основе кластеризации можно выделить группы пациентов с похожими характеристиками")
print("   - K-means с оптимальным k позволяет сегментировать пациентов на группы")
print("   - Эти группы могут помочь в персонализированных рекомендациях по улучшению сна")
print("   - Выявленные кластеры могут соответствовать разным типам расстройств сна")

print("\n" + "="*80)
print("ЗАКЛЮЧЕНИЕ")
print("="*80)

print("\nВ рамках домашнего задания выполнено:")
print("1. ✓ Предобработка данных: One-Hot Encoding категориальных признаков, масштабирование")
print("2. ✓ K-means кластеризация с анализом результатов")
print("3. ✓ Определение оптимального числа кластеров методом локтя")
print("4. ✓ DBSCAN кластеризация с подбором параметров")
print("5. ✓ Сравнение методов и выбор наиболее подходящего")

print(f"\nНаиболее подходящий метод для данного датасета: {'K-means' if 'silhouette_db' not in locals() or silhouette_opt >= silhouette_db else 'DBSCAN'}")
print("\nРезультаты кластеризации сохранены в df_not_processed с колонками:")
print("  - cluster_kmeans_3: K-means с 3 кластерами")
print(f"  - cluster_kmeans_optimal: K-means с оптимальным k={optimal_k}")
print("  - cluster_dbscan: DBSCAN кластеризация")